# Setup a K8s Cluster with Calico

The objective of this notebook is to setup a K8s cluster and a Calico CNI (container network interface) on the [Fabric Testbed](https://portal.fabric-testbed.net/) with the base OS as Ubuntu 22.04.

It refers to the tutorials at:
- [ChatGPT guide](https://docs.google.com/document/d/14d6HMI5jW8NLFe0K4Yx1_bO0DV44ynKMJYQe71ikX7c)
- [Calico quickstart](https://docs.tigera.io/calico/latest/getting-started/kubernetes/quickstart)
- [A video tutorial](https://youtu.be/k3iexxiYPI8)

## Preamble: get a Fabric slice

Our slice contains 3 nodes:
1. The `cpnode` for the [K8s Control Plane](https://kubernetes.io/docs/concepts/overview/components/#control-plane-components).
2. 2 worker nodes for the [K8s Node](https://kubernetes.io/docs/concepts/overview/components/#node-components): `wknode1`, `wknode2`.

### Define the node properties

We configure a L2 network on Fabric so we can manually setup the IPv4 addresses.

In [1]:
# Define the network of the slice
FABRIC_NIC_STR = 'NIC_Basic'  # do not update
FABRIC_SUBNET_STR = "192.168.0.0/24"  # so the node IPs would be 192.168.0.1-10
FABRIC_L2NET_STR = 'site_bridge_net'  # do not update

We need extra storage for the `cpnode`.

In [2]:
# Define the nodes of the slice
node_config = {
    'cpnode': {
        'ip':'192.168.0.1',
        'cores': 8,
        'ram': 24,
        'disk': 100 },
    'wknode1': {
        'ip':'192.168.0.2',
    },
    'wknode2': {
        'ip':'192.168.0.3',
    },
}

### Fabric headers and helper funtions

In [ ]:
from datetime import datetime
from datetime import timezone
from datetime import timedelta

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import ipaddress

import json

fablib = fablib_manager()
fablib.show_config()


Choose the Fabric sites with IPv4 addresses only, such as TOKY and UCSD.

In [ ]:
FABRIC_SITE_OVERRIDE = "TOKY"   # or UCSD, the sites with IPv4 addresses only
FABRIC_SLICENAME_PREFIX = 'k8s_calico_'
FABRIC_OS_STR = 'default_ubuntu_22'

# Define the Fabric slice name with user_id as the suffix
user_info = fablib.get_user_info()
slice_name = FABRIC_SLICENAME_PREFIX + user_info['bastion_login']

# Write selected site into node attributes
for n in node_config:
    node_config[n]['site'] = FABRIC_SITE_OVERRIDE
    node_config[n]['image'] = FABRIC_OS_STR

Build the Fabric slice.

In [ ]:
slice = fablib.new_slice(name=slice_name)

# Create the network
net1 = slice.add_l2network(name=FABRIC_L2NET_STR, subnet=IPv4Network(FABRIC_SUBNET_STR))

In [ ]:
# Create nodes using subnet address assignment
skip_keys = ['ip']
nodes = dict()

for node_name, node_attr in node_config.items():
    print(f"{node_name=}, {node_attr['ip']}")
    nodes[node_name] = slice.add_node(
        name=node_name,
        **{x: node_attr[x] for x in node_attr if x not in skip_keys}
    )
    nic_interface = nodes[node_name].add_component(
        model=FABRIC_NIC_STR,
        name='_'.join([node_name, FABRIC_NIC_STR, 'nic'])
    ).get_interfaces()[0]
    net1.add_interface(nic_interface)
    nic_interface.set_mode('config')
    nic_interface.set_ip_addr(node_attr['ip'])

print(f'Creating a slice named "{slice_name}" with nodes in {FABRIC_SITE_OVERRIDE}')